In [ ]:

import matplotlib
matplotlib.use("PDF")
import pyfits as pf
from pylab import *
import sys, glob
from scipy import stats

thedir='/sandbox/lsst/lsst/GUI/20161010_029-cte/'
%cd $thedir

#%matplotlib inline

In [ ]:
files = glob.glob('ITL-3800C-029_dark_dark_100_20161010??????_bias.fits')
print files

In [ ]:
# [seg, xstart, xpeak, ystart, ystop]
regions = [[8,190,197,754,756],[8,190,197,765,768],[11,490,500,375,386],[11,490,500,402,411]]
figure()
subplots_adjust(hspace = 0.5, wspace = 0.5)
suptitle("CTI - Hot Pixels", fontsize = 18)
plotcounter = 1
for file in files:
    hdulist = pf.open(file, mode='readonly', do_not_scale_image_data=True)
    for [segment,xstart,xpeak,ystart,ystop] in regions:
        xstop = xstart+20
        xaxis = linspace(xstart,xstop-1,20)
        subplot(2,2,plotcounter)
        plotcounter += 1
        imhdr=hdulist[segment+1].header
        extname = imhdr['EXTNAME']
        title("%s, x = %d, y = %d"%(extname,xpeak,(int((ystart+ystop)/2))), fontsize = 10)
        data = array(hdulist[segment+1].data + 32768, dtype = int32)
        cte_data = data[ystart:ystop,:].mean(axis=0)[xstart:xstop]
        cte_std = (data[ystart:ystop,:].std(axis=0) / sqrt(float(ystop-ystart)))[xstart:xstop]
        ratio = data[ystart:ystop,xpeak+1].mean() / data[ystart:ystop,xpeak].mean()
        cti = ratio / xpeak
        scatter(xaxis, cte_data, color = 'blue')
        errorbar(xaxis, cte_data, yerr=cte_std)
        text(xstart-2, 20000, "Ratio = %.3E"%ratio,fontsize=6)
        text(xstart-2, 10000, "CTI = %.3E"%cti,fontsize=6)
        text(xstart,100,"<----",color = 'blue')
        yscale('log',nonposy='clip')
        ylim(.1, 100000)
        xlabel("X")
        ylabel("Signal(ADU)")
savefig("Serial_CTE_Hot_Pixels_12Oct16.pdf")
